In [ ]:
from __future__ import print_function, division
import os
import argparse
import torch.nn as nn
from skimage import io
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from utils import *
from utils.KittiColormap import *
from pyntcloud import PyntCloud
import pandas as pd
from cv_bridge import CvBridge
import cv2
import matplotlib.pyplot as plt
from datasets.data_io import get_transform

## Load Data from rosbag

In [ ]:
import rosbag
bag = rosbag.Bag('zed.bag')
topics = ["/zed2/zed_node/left/camera_info","/zed2/zed_node/left/image_rect_color", "/zed2/zed_node/right/camera_info","/zed2/zed_node/right/image_rect_color"]

In [ ]:
left_image_raw = None
left_camera_info = None
right_image_raw = None
right_camera_info = None
counter = 0
for topic, msg, t in bag.read_messages(topics=topics):
    if counter > 0:
        counter -= 1
        continue
    if (left_image_raw is not None) and (left_camera_info is not None) and (right_image_raw is not None) and (right_camera_info is not None):
        break
    if topic == "/zed2/zed_node/left/image_rect_color":
        left_image_raw = msg
    elif topic == "/zed2/zed_node/left/camera_info":
        left_camera_info = msg
    elif topic == "/zed2/zed_node/right/image_rect_color":
        right_image_raw = msg
    elif topic == "/zed2/zed_node/right/camera_info":
        right_camera_info = msg

In [ ]:
left_camera_info

In [ ]:
right_camera_info

In [ ]:
bridge = CvBridge()
left_image = bridge.imgmsg_to_cv2(left_image_raw, desired_encoding='bgr8')
left_frame = cv2.cvtColor(left_image, cv2.COLOR_BGR2RGB)
left_frame = left_frame[40:,...]
plt.imshow(left_frame)
left_depth_rgb = left_frame[:, :, :3]
depth_rgb = np.transpose(left_depth_rgb, (2, 0, 1))

In [ ]:
left_frame.shape

In [ ]:
right_image = bridge.imgmsg_to_cv2(right_image_raw, desired_encoding='bgr8')
right_frame = cv2.cvtColor(right_image, cv2.COLOR_BGR2RGB)
right_frame = right_frame[40:,...]
plt.imshow(right_frame)
right_depth_rgb = right_frame[:, :, :3]

In [ ]:
left_image.shape

In [ ]:
plt.imshow(cv2.cvtColor(left_frame, cv2.COLOR_BGR2GRAY), 'gray')
plt.show()
plt.imshow(cv2.cvtColor(right_frame, cv2.COLOR_BGR2GRAY), 'gray')
plt.show()
stereo = cv2.StereoSGBM_create(numDisparities=128, blockSize=11)
disparity = stereo.compute(cv2.cvtColor(left_frame, cv2.COLOR_BGR2GRAY),cv2.cvtColor(right_frame, cv2.COLOR_BGR2GRAY))
plt.imshow(disparity,'plasma')

In [ ]:
from models.MSNet3D import MSNet3D
model = MSNet3D(192)
model = nn.DataParallel(model)
model.cuda()

In [ ]:
ckpt_path = "MSNet3D_SF_DS_KITTI2015.ckpt"
print("Loading model {}".format(ckpt_path))
state_dict = torch.load(ckpt_path)
model.load_state_dict(state_dict['model'])

In [ ]:
# Camera intrinsics and extrinsics
c_u = right_camera_info.P[2]
c_v = right_camera_info.P[6]
f_u = right_camera_info.P[0]
f_v = right_camera_info.P[5]
b_x = right_camera_info.P[3] / (-f_u)  # relative
b_y = right_camera_info.P[7] / (-f_v)

In [ ]:
left_frame.shape

In [ ]:
import matplotlib.pyplot as plt
processed = get_transform()
h,w,_ = left_frame.shape

sample_left = processed(left_frame).numpy()
sample_right = processed(right_frame).numpy()

model.eval()
plt.imshow(np.transpose(sample_left, (1,2,0)))
plt.show()
plt.imshow(np.transpose(sample_right, (1,2,0)))
plt.show()

sample_left = torch.Tensor(sample_left)
sample_right = torch.Tensor(sample_right)

sample_left = torch.unsqueeze(sample_left, dim=0)
sample_right = torch.unsqueeze(sample_right, dim=0)

with torch.no_grad():
    disp_est_tn = model(sample_left.cuda(), sample_right.cuda())[0]
    disp_est_np = tensor2numpy(disp_est_tn)
    disp_est = np.array(disp_est_np[0], dtype=np.float32)
    plt.imshow(kitti_colormap(disp_est))
    plt.show()
    disp_est[disp_est < 0] = 0
#             baseline = 0.54
    baseline = b_x
    mask = disp_est > 0
    depth = f_u * baseline / (disp_est + 1. - mask)
    plt.imshow(depth, cmap="plasma")
    plt.show()

In [ ]:
def project_image_to_rect(uv_depth):
    ''' Input: nx3 first two channels are uv, 3rd channel
               is depth in rect camera coord.
        Output: nx3 points in rect camera coord.
    '''
    n = uv_depth.shape[0]
    x = ((uv_depth[:, 0] - c_u) * uv_depth[:, 2]) / f_u + b_x
    y = ((uv_depth[:, 1] - c_v) * uv_depth[:, 2]) / f_v + b_y
    pts_3d_rect = np.zeros((n, 3))
    pts_3d_rect[:, 0] = x
    pts_3d_rect[:, 1] = y
    pts_3d_rect[:, 2] = uv_depth[:, 2]
    return pts_3d_rect

def project_image_to_velo(uv_depth):
    pts_3d_rect = project_image_to_rect(uv_depth)
    return pts_3d_rect

rows, cols = depth.shape
c, r = np.meshgrid(np.arange(cols), np.arange(rows))
points = np.stack([c, r, depth])
points = points.reshape((3, -1))
points = points.T
points = points[mask.reshape(-1)]
cloud = project_image_to_velo(points)
points_rgb = depth_rgb.reshape((3, -1)).T
points_rgb = points_rgb.astype(float)
points_rgb /= 255.

In [ ]:
import open3d as o3d

# Pass xyz to Open3D.o3d.geometry.PointCloud and visualize
rgbd_pcd = o3d.geometry.PointCloud()
rgbd_pcd.points = o3d.utility.Vector3dVector(cloud)
rgbd_pcd.colors = o3d.utility.Vector3dVector(points_rgb)
o3d.io.write_point_cloud("zed.ply", rgbd_pcd)
# rgbd_pcd.paint_uniform_color([1, 0, 0])

# o3d.visualization.draw_geometries([rgbd_pcd],   zoom=0.2599999999999999,
#                                   front=[ 0.19301789659159738, 0.01043594041367546, -0.98113973660383247 ],
#                                   lookat=[ 0.88825355771643766, -0.41312419439599624, 1.606628786781968 ],
#                                   up=[ 0.03817028674385195, -0.99926637964116372, -0.0031195718831270019 ])

In [ ]:
# Pass xyz to Open3D.o3d.geometry.PointCloud and visualize
#zed_pcd = o3d.io.read_point_cloud("zed_point.ply", format="ply")
#o3d.visualization.draw_geometries([zed_pcd],   zoom=0.2599999999999999,
#                                  front=[ 0.19301789659159738, 0.01043594041367546, -0.98113973660383247 ],
#                                  lookat=[ 0.88825355771643766, -0.41312419439599624, 1.606628786781968 ],
#                                  up=[ 0.03817028674385195, -0.99926637964116372, -0.0031195718831270019 ])

In [ ]:
rgbd_voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(rgbd_pcd,
                                                            voxel_size=0.5)

In [ ]:
cloud.shape

In [ ]:
rgbd_voxel_grid

In [ ]:
rgbd_voxel_grid.get_min_bound()

In [ ]:
rgbd_voxel_grid.get_max_bound()

In [ ]:
min_mask = cloud >= [-1.2,-1.0,0.0]
min_mask = min_mask[:, 0] & min_mask[:, 1] & min_mask[:, 2]
max_mask = cloud <= [1.2,0.2,2.4]
max_mask = max_mask[:, 0] & max_mask[:, 1] & max_mask[:, 2]
filter_mask = min_mask & max_mask
filtered_cloud = cloud[filter_mask]
print(filtered_cloud.shape)

In [ ]:
# point_cloud_np = np.asarray([voxel_grid.origin + pt.grid_index*voxel_grid.voxel_size for pt in voxel_grid.get_voxels()])

In [ ]:
filtered_cloud

In [ ]:
voxel_size = 0.05 
xyz_q = np.floor(np.array(filtered_cloud/voxel_size)).astype(int) # quantized point values, here you will loose precision

vox_grid = np.zeros((int(2.4/voxel_size)+1, int(1.2/voxel_size)+1, int(2.4/voxel_size)+1)) #Empty voxel grid

In [ ]:
xyz_q

In [ ]:
offsets = np.array([-xyz_q[:,0].min(), -xyz_q[:,1].min(), -xyz_q[:,2].min()])
print(offsets)

In [ ]:
xyz_offset_q = xyz_q+offsets

In [ ]:
xyz_offset_q

In [ ]:
vox_grid[xyz_offset_q[:,0],xyz_offset_q[:,1],xyz_offset_q[:,2]] = 1 # Setting all voxels containitn a points equal to 1

xyz_v = np.asarray(np.where(vox_grid == 1)) # get back indexes of populated voxels
empty_xyz_v = np.asarray(np.where(vox_grid == 0))

In [ ]:
xyz_v

In [ ]:
cloud_np = np.asarray([(pt-offsets)*voxel_size for pt in xyz_v.T])
empty_cloud_np = np.asarray([(pt-offsets)*voxel_size for pt in empty_xyz_v.T])
print(empty_cloud_np)

In [ ]:
empty_cloud_np.min()

In [ ]:
np_pcd = o3d.geometry.PointCloud()
np_pcd.points = o3d.utility.Vector3dVector(cloud_np)
np_pcd.paint_uniform_color([0.7, 0, 0])

empty_np_pcd = o3d.geometry.PointCloud()
empty_np_pcd.points = o3d.utility.Vector3dVector(empty_cloud_np)
empty_np_pcd.paint_uniform_color([0.0, 0.7, 0])

unit_np_pcd = o3d.geometry.PointCloud()
unit_np_pcd.points = o3d.utility.Vector3dVector(np.array([[0.0, 0.0, 0.0],[0.0, 2.5, 0.0],[0.0, -1.0, 0.0]]))
unit_np_pcd.paint_uniform_color([0.0, 0.0, 1.0])

In [ ]:
o3d.visualization.draw_geometries([rgbd_pcd, np_pcd], zoom=0.65999999999999992,
                                  front=[ -0.014497331658126618, -0.12180282531039557, -0.99244843650499026 ],
                                  lookat=[ 0.48829133058982988, -0.41076821248406992, 2.5847910404205323 ],
                                  up=[ 0.037670557241263204, -0.99191462942433895, 0.1211870333456067 ])